In [211]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd
import time
from random import randint
import numpy as np


In [622]:
 browser = webdriver.Chrome('C:/Users/utilisateur/chromedriver/chromedriver.exe') 

In [576]:
def scrapping_lacentrale (car,pages, csv):
    
#     15 resultat par pages
    
    #url du site lacentrale
   
    url = f'https://www.lacentrale.fr/occasion-voiture-marque-{car}.html'

    
    browser.get(url)
    browser.maximize_window()
    time.sleep(2)

#     creation dataframe 
#     ouvrir fichier csv pour verifier si l'annonce qu'on scrappe n'est pas deja scrappé precedement 
    try:
        df= pd.read_csv(csv)
    except:
        df=pd.DataFrame(columns= [item[0] for item in xpath.items()])
    

    j=0
    while j < pages:
        
        #  Recuperation de tous les href des annonces de la page
        list_link_annonces = browser.find_elements_by_xpath("//div[@class='adLineContainer']/div/a")  
        for i in range(len(list_link_annonces)):
                
                list_link_annonces[i].send_keys(Keys.CONTROL + Keys.RETURN)
                time.sleep(1)
                
                #swich sur le nouveau onglet
                browser.switch_to.window(browser.window_handles[1])
                time.sleep(1)
#                 
                url_annonce=browser.current_url
                browser1=browser.get(url_annonce)
                
                # utilisation fonction infos_annonce pour recuperer les informations
                annonce={}
                
                annonce = infos_annonce(annonce, browser1)
#                
                
                time.sleep(2)
                
                # verification si on deja l'offre (la ref) deja dans la df
                
                try:
                    ref = annonce['reference_annonce']
                    
                except: 
                    ref='pas de ref'
                
                if df[df.reference_annonce == ref].empty:
                    
                    df= df.append(annonce, ignore_index=True)
                    df.to_csv('lacentrale{}.csv'.format(csv), index=False,encoding='utf-8')
                else:
                    print('Ref deja dispo: - ref {} - url {}'.format(ref, url_annonce)) 
                    
#               print(df)
                
                # fermeture e l'onglet et retour 
                browser.close()
                browser.switch_to.window(browser.window_handles[0])
                
                
        
         # page suivant :    
        try:
            nextpage_button = browser.find_element_by_css_selector("[title='Page suivante']")  
            time.sleep(2)
            ActionChains(browser).move_to_element(nextpage_button) 
            time.sleep(2)
            ActionChains(browser).click(nextpage_button).perform() 
            j+=1

        except (NoSuchElementException) :
            print(":p")
            break
       
    return df

In [15]:
xpath={
"nom":"//div[@class='cbm-sideBar']/div/div/div",
"marque":"//div[@class='cbm-price']/div/div/a",
"modele":"//div[@class='cbm-price']/div/div/a",
"categorie":"//section[@class='containerSite containerContent']/div/div/div/div/div/nav/ol/li[2]/a/span",
"Emission_CO2": "//span[contains(@class, 'cbm__bilan--co2')]",
"kilometrage":"//span[preceding::button[text()[contains(.,'Kilométrage compteur')]]]",
"energie":"//span[preceding::button[text()[contains(.,'Énergie')]]]",
"boite_de_vitesse":"//span[preceding::button[text()[contains(.,'Boîte de vitesse')]]]", 
"couleur":"//span[preceding::button[text()[contains(.,'Couleur extérieure')]]]",
"nombre_portes":"//span[preceding::button[text()[contains(.,'Nombre de portes')]]]",
"nombre_place":"//span[preceding::button[text()[contains(.,'Nombre de places')]]]",
"mise_circulation":"//span[preceding::button[text()[contains(.,'Mise en circulation')]]]",
"reference_annonce":"//section[@id='generalInformation']/div[1]/span",
"conso_mixte":"//li[contains(text(),'Consommation mixte')]",
"Puissance_fiscale":"//span[preceding::button[text()[contains(.,'Puissance fiscale')]]]",
"Puissance_din":"//span[preceding::button[text()[contains(.,'Puissance din')]]]",
"Crit_Air":'//span[preceding::button[text()[contains(.,"Crit\'Air")]]]',
"prix":"//span[@class='cbm__priceWrapper']"
}




In [16]:
def get_element(key,xpath, annonce, browser1):
    
    print(browser1)
    browser1
    
    if key== 'marque':
        try:
            annonce[key]= browser.find_element_by_xpath(xpath[key]).get_attribute('data-make')
        except:
            pass 
    elif key== 'modele':
        try:
            annonce[key]= browser.find_element_by_xpath(xpath[key]).get_attribute('data-model')
        except:
            pass 
    else:    
        try:
            annonce[key]= browser.find_element_by_xpath(xpath[key]).text
        except:
            pass
    
                
    return annonce

        

def infos_annonce(annonce,browser1):
        annonce= get_element('nom',xpath,annonce,browser1)
        annonce= get_element('marque',xpath,annonce,browser1)
        annonce= get_element('modele',xpath,annonce,browser1)
        annonce= get_element('categorie',xpath,annonce,browser1)
        annonce= get_element('Emission_CO2',xpath,annonce,browser1)
        annonce= get_element('kilometrage',xpath,annonce,browser1)
        annonce= get_element('energie',xpath,annonce,browser1)
        annonce= get_element('boite_de_vitesse',xpath,annonce,browser1)
        annonce= get_element('couleur',xpath,annonce,browser1)
        annonce= get_element('nombre_portes',xpath,annonce,browser1)
        annonce= get_element('nombre_place',xpath,annonce,browser1)
        annonce= get_element('mise_circulation',xpath,annonce,browser1)
        annonce= get_element('reference_annonce',xpath,annonce,browser1)
        annonce= get_element('conso_mixte',xpath,annonce,browser1)
        annonce= get_element('Puissance_fiscale',xpath,annonce,browser1)
        annonce= get_element('Puissance_din',xpath,annonce,browser1)
        annonce= get_element('Crit_Air',xpath,annonce,browser1)
        annonce= get_element('prix',xpath,annonce,browser1)
   
        return annonce


In [624]:
# df_peugeot= scrapping_lacentrale('peugeot',50, 'tt')


In [18]:
# df_renault= scrapping_lacentrale('renault',50)

In [19]:
# df_citroen=scrapping_lacentrale('citroen',50)

In [20]:
def to_csv(df):
    try:
        df.to_csv('lacentrale.csv', index=False,encoding='utf-8', mode='a')
    except:
        df.to_csv('lacentrale.csv', index=False,encoding='utf-8')


In [625]:
scrapping = pd.read_csv('lacentrale2.csv')
scrapping

,nom,marque,modele,categorie,Emission_CO2,kilometrage,energie,boite_de_vitesse,couleur,nombre_portes,nombre_place,mise_circulation,reference_annonce,conso_mixte,Puissance_fiscale,Puissance_din,Crit_Air,prix
0,PEUGEOT 2008,PEUGEOT,2008,"4x4, SUV & Crossover occasion",A,64 464 Km,Diesel,automatique,gris foncé metal,5.0,5.0,25/12/2018,Réf. pro : 81 | Réf. annonce : W101736655,Consommation mixte : 3 L,5 CV,99 ch,2.0,18 120 €
1,PEUGEOT 308 (2E GENERATION),PEUGEOT,308 (2E GENERATION),Berline occasion,A,49 131 Km,Diesel,mécanique,gris foncé metal,5.0,5.0,20/08/2019,Réf. pro : 90 | Réf. annonce : W101742542,Consommation mixte : 3 L,6 CV,131 ch,2.0,19 420 €
2,PEUGEOT EXPERT 3 FOURGON,PEUGEOT,EXPERT 3 FOURGON,"Fourgon (moins de 3,5 t) occasion",C,35 591 Km,Diesel,mécanique,blanc verni,4.0,3.0,03/01/2020,Réf. pro : 33 | Réf. annonce : W101725570,Consommation mixte : 5 L,7 CV,122 ch,2.0,19 150 €
3,PEUGEOT 308 (2E GENERATION) SW,PEUGEOT,308 (2E GENERATION) SW,Break occasion,A,105 452 Km,Diesel,mécanique,blanc metal,5.0,5.0,10/04/2017,Réf. pro : 86 | Réf. annonce : W101739444,Consommation mixte : 3 L,5 CV,99 ch,2.0,15 600 €
4,PEUGEOT 5008,PEUGEOT,5008,Monospace occasion,B,55 000 Km,Diesel,mécanique,blanc nacre,5.0,7.0,01/07/2017,Réf. annonce : W101742031,Consommation mixte : 4 L,8 CV,150 ch,2.0,21 750 €
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,PEUGEOT 308 (2E GENERATION),PEUGEOT,308 (2E GENERATION),Berline occasion,B,12 545 Km,Essence,mécanique,gris,5.0,5.0,04/02/2020,Réf. pro : 11571738 | Réf. annonce : E106156349,Consommation mixte : 4.6 L,5 CV,110 ch,1.0,19 020 €
376,PEUGEOT 308,PEUGEOT,308,Berline occasion,B,69 019 Km,Diesel,mécanique,VAPOR GREY,5.0,5.0,23/06/2013,Réf. pro : qgrc_405 | Réf. annonce : E18971665,Consommation mixte : 4.2 L,6 CV,112 ch,2.0,12 700 €
377,PEUGEOT 108,PEUGEOT,108,Citadine occasion,A,74 238 Km,Essence,mécanique,Blanc Lipizan,5.0,4.0,23/09/2017,Réf. pro : dubr_104543 | Réf. annonce : E10606...,Consommation mixte : 4.3 L,4 CV,82 ch,1.0,10 470 €
378,PEUGEOT 2008 (2E GENERATION),PEUGEOT,2008 (2E GENERATION),"4x4, SUV & Crossover occasion",B,1 060 Km,Essence,mécanique,Rouge Métal,5.0,5.0,04/07/2020,Réf. pro : pe44a5_4940 | Réf. annonce : E10582...,Consommation mixte : 4.5 L,6 CV,130 ch,1.0,26 480 €


In [390]:
def clean_scrapping_centrale(df):
        df['prix'] = df['prix'].str.replace('€', '', regex=True)
        df['prix'] = df['prix'].str.replace(' ', '', regex=True)
        
        df['kilometrage'] = df['kilometrage'].str.replace('Km', '', regex=True)
        df['kilometrage'] = df['kilometrage'].str.replace(' ', '', regex=True)
        
        df['conso_mixte'] = df['conso_mixte'].str.replace('Consommation mixte :', '', regex=True)
        df['conso_mixte'] = df['conso_mixte'].str.replace('L', '', regex=True)
        df['conso_mixte'] = df['conso_mixte'].str.strip()
        
        df['categorie'] = df['categorie'].str.replace('occasion', '', regex=True)
        df['categorie'] = df['categorie'].str.strip()
        
        df['Puissance_fiscale'] = df['Puissance_fiscale'].str.replace('CV', '', regex=True)
        df['Puissance_fiscale'] = df['Puissance_fiscale'].str.strip()
        
        df['Puissance_din'] = df['Puissance_din'].str.replace('ch', '', regex=True)
        df['Puissance_din'] = df['Puissance_din'].str.strip()
        
        col_name= ['nom','marque', 'modele','categorie','co2','nb_km','carburant', 'boite_vitesse', 'couleur','nb_portes',
       'nb_places', 'mise_circulation','ref', 'conso_mixte', 'p_fiscale', 'p_din',
       'critair', 'prix']

        df.columns = col_name
        
        df= df[['nom', 'ref', 'marque', 'modele', 'categorie', 'co2',
       'nb_km', 'carburant', 'boite_vitesse', 'couleur', 'nb_portes',
       'nb_places', 'mise_circulation', 'conso_mixte', 'p_fiscale', 'p_din',
       'critair', 'prix']]
        
#         for col in [ 'nb_km', 'nb_portes', 'nb_places', 'p_fiscale', 'p_din', 'critair', 'prix']:
#                 df[col] = df[col].astype(int)
        
        df.drop(df[(df.categorie =='Fourgon (moins de 3,5 t)') | (df.categorie =='Fourgonnette') |
                    (df.categorie =='Break')].index, inplace=True)
        
        df.drop(df[(df.categorie =='Voiture société') | (df.categorie =='Coupé') | (df.categorie =='Cabriolet') |
                    (df.categorie =='Bus et minibus') | (df.categorie =='Voiture société, commerciale') ].index, inplace=True)
        

        
        df.drop(df[(df.carburant =='Hybride diesel électrique') | (df.carburant =='Electrique') |
                    (df.carburant =='Hybride diesel électrique')].index, inplace=True)
        
        df.drop(df[(df.carburant =='Hybride diesel électrique') | (df.carburant =='Electrique') |
                    (df.carburant =='Hybride diesel électrique')].index, inplace=True)
            
        df['categorie'].replace({'4x4, SUV & Crossover':'SUV 4X4','Citadine':'CITADINE','Monospace':'MONOSPACE','Berline':'BERLINE'},inplace=True)
        df['carburant'].replace({'Diesel':'DIESEL','Essence':'ESSENCE'},inplace=True)
        df['boite_vitesse'].replace({'mécanique':'MANUELLE','automatique':'AUTO'},inplace=True)
        df['mise_circulation'] = df['mise_circulation'].str.extract('(\d{4})').astype(int)
        
        df['modele'] = df['modele'].str.replace('\((.*)\)', '', regex=True)
        df['modele'] = df['modele'].str.strip()
        df['mise_circulation'] = df['mise_circulation'].str.replace('2022', '2020', regex=True)
        df['mise_circulation'] = df['mise_circulation'].str.replace('2021', '2020', regex=True)
        
        df['mise_circulation'] = df['mise_circulation'].astype(str)
        
        for i in [x for x in df.columns if x not in ['nb_km', 'nb_portes', 'nb_places', 'p_fiscale', 'p_din', 'critair', 'prix']]:
            df[i] = df[i].astype(str)
            df[i] = df[i].str.lower()
    
    
        for i in [x for x in df.columns if x not in ['nb_km', 'nb_portes', 'nb_places', 'p_fiscale', 'p_din', 'critair', 'prix']]:
            df[i] = df[i].str.capitalize()  
        
        
        return df 

In [562]:
# dataset

,nom,ref,marque,modele,categorie,co2,nb_km,carburant,boite_vitesse,couleur,nb_portes,nb_places,mise_circulation,conso_mixte,p_fiscale,p_din,critair,prix
0,Renault scenic 3,Réf. pro : 120245 | réf. annonce : e106531159,Renault,Scenic 3,Monospace,C,164697,Diesel,Manuelle,Gris,5.0,5.0,2010,4.9,6,106,3.0,6130
1,Renault clio 4,Réf. pro : 50 | réf. annonce : w101726585,Renault,Clio 4,Citadine,B,21830,Diesel,Manuelle,Gris foncé metal,5.0,5.0,2019,4,5,90,2.0,15220
2,Renault clio 4,Réf. pro : 59 | réf. annonce : w101726537,Renault,Clio 4,Citadine,A,62966,Diesel,Manuelle,Blanc verni,5.0,5.0,2017,3,4,90,2.0,12300
3,Renault kadjar,Réf. pro : 65 | réf. annonce : w101728160,Renault,Kadjar,Suv 4x4,B,12,Diesel,Auto,Bleu metal,5.0,5.0,2020,4.3,6,115,2.0,26700
4,Renault megane 3,Réf. pro : 105 | réf. annonce : w101746629,Renault,Megane 3,Berline,B,95646,Diesel,Manuelle,Blanc verni,5.0,5.0,2015,4,5,95,2.0,9840
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1458,Peugeot 3008 (2e generation),Réf. pro : jonz_3008alb130dnj | réf. annonce :...,Peugeot,3008,Suv 4x4,B,10069,Diesel,Manuelle,Noir perla nera,5.0,5.0,2020,4,6,130,2.0,32720
1459,Peugeot 5008 (2e generation),Réf. annonce : w101738656,Peugeot,5008,Monospace,B,11,Essence,Auto,Noir metal,5.0,7.0,2020,5.1,6,130,1.0,39770
1460,Peugeot 508 (2e generation),Réf. pro : ft33c1_bsa707994 | réf. annonce : e...,Peugeot,508,Berline,B,21115,Diesel,Auto,Gris platinium,5.0,5.0,2020,4.5,8,160,2.0,33490
1461,Peugeot 208,Réf. pro : 170625 | réf. annonce : e106496463,Peugeot,208,Citadine,B,18837,Essence,Manuelle,Blanc,5.0,5.0,2020,4.7,4,83,1.0,14610


In [575]:
# modele beke pour la database flask 

,id,modele,marque,categorie
0,0,108,Peugeot,Citadine
1,1,2008,Peugeot,Suv 4x4
2,2,208,Peugeot,Citadine
3,3,3008,Peugeot,Suv 4x4
4,4,308,Peugeot,Berline
5,5,308 sw,Peugeot,Berline
6,6,5008,Peugeot,Suv 4x4
7,7,508,Peugeot,Berline
8,8,508 nouvelle,Peugeot,Berline
9,9,Berlingo,Citroën,Monospace


In [583]:
# csv to sql
import sqlite3
import pandas as pd

# load data
df = pd.read_csv('bekefinale.csv')
# strip whitespace from headers
df.columns = df.columns.str.strip()

con = sqlite3.connect("database2.db")

# drop data into database
df.to_sql("Voiture", con)

con.close()